In [42]:
import numpy as np
import pandas as pd
import chardet
import datetime
from datetime import timedelta
import os

## Read File

In [43]:
# Detect the encoding of the file
with open('newleads.csv', 'rb') as f:
    result = chardet.detect(f.read())

# Read the file using the detected encoding
df = pd.read_csv('newleads.csv', encoding=result['encoding'])

In [44]:
df.head()

,Filed As,Status,CreatedOn,Source,Feedback Score,Country,Registration #,Registration Type,Employment Preference Attributes,Specialty,Seniority,Primary Consultant,Primary Email
0,Test Test,New Lead,01/09/2023 14:09,Referral - Personal referral,NaN,Australia,NaN,NaN,Locum,Emergency Medicine (A&E),Director / Clinical Director,NaN,NaN
1,Test Test,New Lead,01/09/2023 14:13,Referral - Personal referral,NaN,Australia,NaN,NaN,Locum,Emergency Medicine (A&E),Director / Clinical Director,NaN,NaN
2,Test Test,New Lead,01/09/2023 14:15,Referral - Personal referral,NaN,Australia,NaN,NaN,Locum,Emergency Medicine (A&E),Director / Clinical Director,NaN,NaN
3,Test Test,New Lead,01/09/2023 14:26,Referral - Personal referral,NaN,Australia,NaN,NaN,Locum,Emergency Medicine (A&E),Director / Clinical Director,NaN,NaN
4,Mattias Test,New Lead,24/10/2023 8:57,Google,NaN,Australia,NaN,NaN,"Locum, Permanent",General Practice,General Practitioner,NaN,NaN


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1621 entries, 0 to 1620
Data columns (total 13 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Filed As                          1621 non-null   object 
 1   Status                            1621 non-null   object 
 2   CreatedOn                         1621 non-null   object 
 3   Source                            1621 non-null   object 
 4   Feedback Score                    0 non-null      float64
 5   Country                           1620 non-null   object 
 6   Registration #                    407 non-null    object 
 7   Registration Type                 404 non-null    object 
 8   Employment Preference Attributes  1621 non-null   object 
 9   Specialty                         1619 non-null   object 
 10  Seniority                         1621 non-null   object 
 11  Primary Consultant                223 non-null    object 
 12  Primar

## Remove Test

In [46]:
# Define substrings to filter
substrings_to_remove = ['test', 'graham long', 'lizzie new' , 'Tahnee Love']

# Create a boolean mask to filter rows
mask = ~df['Filed As'].str.lower().str.contains('|'.join(substrings_to_remove), case=False)

# Apply the mask to the DataFrame
df = df[mask]

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1608 entries, 5 to 1620
Data columns (total 13 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Filed As                          1608 non-null   object 
 1   Status                            1608 non-null   object 
 2   CreatedOn                         1608 non-null   object 
 3   Source                            1608 non-null   object 
 4   Feedback Score                    0 non-null      float64
 5   Country                           1607 non-null   object 
 6   Registration #                    407 non-null    object 
 7   Registration Type                 404 non-null    object 
 8   Employment Preference Attributes  1608 non-null   object 
 9   Specialty                         1608 non-null   object 
 10  Seniority                         1608 non-null   object 
 11  Primary Consultant                223 non-null    object 
 12  Primary Ema

In [48]:
#remove RMS
# List of consultants to remove
consultants_to_remove = ['Leiara Ferrett', 'Ebony Kouka', 'Alexandra Graham', 'Lisa Greaves', 'Elinor Faulkner']

# Filter out rows where Primary Consultant is in the list of consultants to remove
df = df[~df['Primary Consultant'].isin(consultants_to_remove)]

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1602 entries, 5 to 1620
Data columns (total 13 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Filed As                          1602 non-null   object 
 1   Status                            1602 non-null   object 
 2   CreatedOn                         1602 non-null   object 
 3   Source                            1602 non-null   object 
 4   Feedback Score                    0 non-null      float64
 5   Country                           1601 non-null   object 
 6   Registration #                    407 non-null    object 
 7   Registration Type                 404 non-null    object 
 8   Employment Preference Attributes  1602 non-null   object 
 9   Specialty                         1602 non-null   object 
 10  Seniority                         1602 non-null   object 
 11  Primary Consultant                217 non-null    object 
 12  Primary Ema

In [50]:
# Filter condition to remove specific emails from 'Primary Email'
condition = (df['Primary Email'].str.endswith('@wave.com.au')) & (~df['Primary Email'].isin(['unknown@wave.com.au', 'tbc@wave.com.au']))

# Removing rows based on the condition
df = df[~condition]

# Dropping the 'Primary Email' column
df = df.drop('Primary Email', axis=1)

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1599 entries, 5 to 1620
Data columns (total 12 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Filed As                          1599 non-null   object 
 1   Status                            1599 non-null   object 
 2   CreatedOn                         1599 non-null   object 
 3   Source                            1599 non-null   object 
 4   Feedback Score                    0 non-null      float64
 5   Country                           1599 non-null   object 
 6   Registration #                    407 non-null    object 
 7   Registration Type                 404 non-null    object 
 8   Employment Preference Attributes  1599 non-null   object 
 9   Specialty                         1599 non-null   object 
 10  Seniority                         1599 non-null   object 
 11  Primary Consultant                215 non-null    object 
dtypes: float64(

In [52]:
# Create a boolean mask to filter rows
mask = ~((df['Status'] == 'Imported') & (df['Source'].isin(['Headhunt', 'Linkedin'])))

# Apply the mask to the original DataFrame 'df'
df = df[mask]

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1489 entries, 5 to 1620
Data columns (total 12 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Filed As                          1489 non-null   object 
 1   Status                            1489 non-null   object 
 2   CreatedOn                         1489 non-null   object 
 3   Source                            1489 non-null   object 
 4   Feedback Score                    0 non-null      float64
 5   Country                           1489 non-null   object 
 6   Registration #                    322 non-null    object 
 7   Registration Type                 319 non-null    object 
 8   Employment Preference Attributes  1489 non-null   object 
 9   Specialty                         1489 non-null   object 
 10  Seniority                         1489 non-null   object 
 11  Primary Consultant                193 non-null    object 
dtypes: float64(

## add week number

In [54]:
# Function to calculate the week number based on weeks ending on Sundays
def calculate_week_monday_start(date):
    start_date = pd.to_datetime('2023-07-01')
    days_since_start = (date - start_date).days

    # Adjusting for a Monday start
    offset = (start_date.dayofweek - 0) % 7
    days_since_start += offset

    if days_since_start < 0:
        return 0  # If the date is before the start date, return 0 or handle as needed
    else:
        if days_since_start <= 7:
            return 1
        else:
            week_number = 1 + ((days_since_start - 7) // 7)  # Starting from week 2
            return week_number

# Assuming 'df' is your DataFrame and 'CreatedOn' is your date field
df['CreatedOn'] = pd.to_datetime(df['CreatedOn'], format='%d/%m/%Y %H:%M', errors='coerce')

# Apply the function to calculate the week number
df['FY24 Week'] = df['CreatedOn'].apply(calculate_week_monday_start)

In [55]:
df.sample(5)

,Filed As,Status,CreatedOn,Source,Feedback Score,Country,Registration #,Registration Type,Employment Preference Attributes,Specialty,Seniority,Primary Consultant,FY24 Week
1288,Jiz Mary Santhosh,Transitional,2023-10-25 12:24:00,Email,NaN,India,NaN,NaN,Permanent,Surgery - Ophthalmology,Consultant / Specialist,NaN,17
165,Mohammad Nasir Iqbal,Live,2023-07-14 12:43:00,Email,NaN,South Africa,NaN,NaN,Permanent,Surgery - Orthopaedics/TR,Consultant / Specialist,NaN,2
826,Susmita Shaha,Transitional,2023-09-24 18:10:00,Indeed,NaN,Australia,NaN,NaN,Locum,Emergency Medicine (A&E),Resident (RMO),NaN,12
430,Opobo Joel,Ineligible,2023-08-14 05:15:00,Indeed,NaN,Uganda,NaN,NaN,Permanent,"Surgery - Orthopaedics/TR, Surgery - Oral Maxi...",General Practitioner,NaN,7
113,Abinaya Murali,Live,2023-07-10 23:15:00,Paid Google,NaN,United Kingdom,NaN,NaN,Permanent,Obstetrics & Gynaecology,"Registrar, Resident (RMO), SMO",NaN,2


## Map Team Allocated

In [56]:
# Create a dictionary to map 'Primary Consultant' to 'team allocated'
team_mapping = {
    'Tahnee Love': 'JLD',
    'Craig Picard': 'JLD',
    'Ben Chegwidden': 'JLD',
    'Fiona Jackson': 'JLD',
    'Aimee Skoyles': 'JLD',
    'Mikaila Brooks': 'JLD',
    'Robyn Pascoe': 'Psych',
    'Courtney Lewis': 'Psych',
    'Yasmin Lockey': 'Psych',
    'Caitlin Lingard': 'HS Perm',
    'Jennifer Salinas': 'HS Perm',
    'Ashlea Harvey': 'HS Perm',
    'Chloe Frost': 'HS Perm',
    'EJ Cuaresma': 'GP Perm',
    'Jane Stanke': 'GP Perm',
    'Amber Derby-Davies': 'HS Locum',
    'Jade Camilleri': 'HS Locum',
    'Claudine Zaarour': 'HS Locum',
    'Hannah Kearns': 'HS Locum',
    'Eamon McCurry': 'HS Locum',
    'Nicole Langan': 'HS Locum',
    'Amy Beddall': 'GP Locum',
    'Cory Robertson': 'GP Locum',
    'Charlotte Hellmundt': 'GP Locum',
    'Anna Mullins': 'GP Locum',
    'Jade-Maree Camilleri': 'HS Locum',
    'Xanthia Gardner': 'HS Perm',
    'Charlie Hellmundt': 'GP Locum',
    'Paul Campbell': 'GP Locum',
    'Sharlina Drutschmann': 'GP Locum',
}

# Create the 'team allocated' column based on 'Primary Consultant' using the mapping dictionary
df['Team Allocated'] = df['Primary Consultant'].map(team_mapping)


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1489 entries, 5 to 1620
Data columns (total 14 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Filed As                          1489 non-null   object        
 1   Status                            1489 non-null   object        
 2   CreatedOn                         1489 non-null   datetime64[ns]
 3   Source                            1489 non-null   object        
 4   Feedback Score                    0 non-null      float64       
 5   Country                           1489 non-null   object        
 6   Registration #                    322 non-null    object        
 7   Registration Type                 319 non-null    object        
 8   Employment Preference Attributes  1489 non-null   object        
 9   Specialty                         1489 non-null   object        
 10  Seniority                         1489 non-null   obj

In [58]:
# Filter for unmapped rows (where 'Primary Consultant' is not null and 'Team Allocated' is missing)
unmapped_rows = df[df['Primary Consultant'].notna() & df['Team Allocated'].isna()]

# Get the unique values of 'Primary Consultant' for unmapped rows
unique_primary_consultants = unmapped_rows['Primary Consultant'].unique()

# Display the unique values
print(unique_primary_consultants)

['Carole Paterson' 'Wave Info']


In [59]:
# Non-Australia - Assign 'HS Perm' or 'GP Perm' based on specified conditions
non_australia = (df['Country'] != 'Australia')

# Check if neither 'Specialty' nor 'Seniority' contains 'GP' related phrases
hs_perm_conditions = (
    non_australia &
    ~(
        (df['Specialty'].str.contains('GP|General Practice|General Practitioner', case=False)) | 
        (df['Seniority'].str.contains('GP|General Practice|General Practitioner', case=False))
    )
)

# Update 'HS Perm' for null 'Team Allocated' where the conditions are met
df.loc[hs_perm_conditions & df['Team Allocated'].isnull(), 'Team Allocated'] = 'HS Perm'

# Check for 'GP Perm' based on the same conditions
gp_perm_conditions = (
    non_australia &
    (
        (df['Specialty'].str.contains('GP|General Practice|General Practitioner', case=False)) | 
        (df['Seniority'].str.contains('GP|General Practice|General Practitioner', case=False))
    )
)

# Update 'GP Perm' for null 'Team Allocated' based on specified conditions
df.loc[gp_perm_conditions & df['Team Allocated'].isnull(), 'Team Allocated'] = 'GP Perm'


In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1489 entries, 5 to 1620
Data columns (total 14 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Filed As                          1489 non-null   object        
 1   Status                            1489 non-null   object        
 2   CreatedOn                         1489 non-null   datetime64[ns]
 3   Source                            1489 non-null   object        
 4   Feedback Score                    0 non-null      float64       
 5   Country                           1489 non-null   object        
 6   Registration #                    322 non-null    object        
 7   Registration Type                 319 non-null    object        
 8   Employment Preference Attributes  1489 non-null   object        
 9   Specialty                         1489 non-null   object        
 10  Seniority                         1489 non-null   obj

In [61]:
#Psych
psych_conditions = (
    (df['Country'] == 'Australia') &
    (df['Specialty'].str.startswith('Psych')) &
    (df['Team Allocated'].isnull())
)

# Set 'Psych' for 'Team Allocated' if conditions are met
df.loc[psych_conditions, 'Team Allocated'] = 'Psych'

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1489 entries, 5 to 1620
Data columns (total 14 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Filed As                          1489 non-null   object        
 1   Status                            1489 non-null   object        
 2   CreatedOn                         1489 non-null   datetime64[ns]
 3   Source                            1489 non-null   object        
 4   Feedback Score                    0 non-null      float64       
 5   Country                           1489 non-null   object        
 6   Registration #                    322 non-null    object        
 7   Registration Type                 319 non-null    object        
 8   Employment Preference Attributes  1489 non-null   object        
 9   Specialty                         1489 non-null   object        
 10  Seniority                         1489 non-null   obj

In [63]:
#Australia Perm
employment_conditions = (
    (df['Country'] == 'Australia') &
    (~df['Employment Preference Attributes'].str.contains('Locum', case=False))
)

gp_perm_conditions = (
    employment_conditions &
    (
        (df['Specialty'].str.contains('GP|General Practice|General Practitioner', case=False)) | 
        (df['Seniority'].str.contains('GP|General Practice|General Practitioner', case=False))
    )
)

# Set 'GP Perm' for 'Team Allocated' based on conditions
df.loc[gp_perm_conditions & df['Team Allocated'].isnull(), 'Team Allocated'] = 'GP Perm'

# Set 'HS Perm' for remaining null 'Team Allocated' based on conditions
df.loc[employment_conditions & df['Team Allocated'].isnull(), 'Team Allocated'] = 'HS Perm'

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1489 entries, 5 to 1620
Data columns (total 14 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Filed As                          1489 non-null   object        
 1   Status                            1489 non-null   object        
 2   CreatedOn                         1489 non-null   datetime64[ns]
 3   Source                            1489 non-null   object        
 4   Feedback Score                    0 non-null      float64       
 5   Country                           1489 non-null   object        
 6   Registration #                    322 non-null    object        
 7   Registration Type                 319 non-null    object        
 8   Employment Preference Attributes  1489 non-null   object        
 9   Specialty                         1489 non-null   object        
 10  Seniority                         1489 non-null   obj

In [65]:
#Junior
jld_conditions = (
    df['Team Allocated'].isnull() &
    df['Seniority'].str.contains('resident|registrar|cmo / smo|intern|student', case=False)
)

# Set 'JLD' for 'Team Allocated' where conditions are met
df.loc[jld_conditions, 'Team Allocated'] = 'JLD'

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1489 entries, 5 to 1620
Data columns (total 14 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Filed As                          1489 non-null   object        
 1   Status                            1489 non-null   object        
 2   CreatedOn                         1489 non-null   datetime64[ns]
 3   Source                            1489 non-null   object        
 4   Feedback Score                    0 non-null      float64       
 5   Country                           1489 non-null   object        
 6   Registration #                    322 non-null    object        
 7   Registration Type                 319 non-null    object        
 8   Employment Preference Attributes  1489 non-null   object        
 9   Specialty                         1489 non-null   object        
 10  Seniority                         1489 non-null   obj

In [67]:
#Australia Locum
# Check if either 'Specialty' or 'Seniority' contains GP-related phrases
gp_locum_conditions = (
    (df['Team Allocated'].isnull()) &
    (df['Specialty'].str.contains('GP|General Practitioner|General Practice', case=False) |
    df['Seniority'].str.contains('GP|General Practitioner|General Practice', case=False))
)

# Set 'GP Locum' for rows with null 'Team Allocated' based on the specified conditions
df.loc[gp_locum_conditions, 'Team Allocated'] = 'GP Locum'

# Set 'HS Locum' for the remaining null 'Team Allocated' rows that don't meet the GP conditions
df.loc[df['Team Allocated'].isnull(), 'Team Allocated'] = 'HS Locum'

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1489 entries, 5 to 1620
Data columns (total 14 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Filed As                          1489 non-null   object        
 1   Status                            1489 non-null   object        
 2   CreatedOn                         1489 non-null   datetime64[ns]
 3   Source                            1489 non-null   object        
 4   Feedback Score                    0 non-null      float64       
 5   Country                           1489 non-null   object        
 6   Registration #                    322 non-null    object        
 7   Registration Type                 319 non-null    object        
 8   Employment Preference Attributes  1489 non-null   object        
 9   Specialty                         1489 non-null   object        
 10  Seniority                         1489 non-null   obj

In [69]:
#diveregent  careers/ executives
# Condition to check Seniority and Country
seniority_conditions = (
    (df['Seniority'].str.contains('Director|DMS|Health Executive|Divergent Careers|Allied Health|Medical Admin|Scientist', case=False, na=False)) &
    (df['Country'] == 'Australia')
)

# Update 'HS Perm' for the specified conditions
df.loc[seniority_conditions, 'Team Allocated'] = 'HS Perm'

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1489 entries, 5 to 1620
Data columns (total 14 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Filed As                          1489 non-null   object        
 1   Status                            1489 non-null   object        
 2   CreatedOn                         1489 non-null   datetime64[ns]
 3   Source                            1489 non-null   object        
 4   Feedback Score                    0 non-null      float64       
 5   Country                           1489 non-null   object        
 6   Registration #                    322 non-null    object        
 7   Registration Type                 319 non-null    object        
 8   Employment Preference Attributes  1489 non-null   object        
 9   Specialty                         1489 non-null   object        
 10  Seniority                         1489 non-null   obj

In [71]:
# Check for the specified condition
condition = (df['Seniority'].str.contains('Director|DMS|Health Executive|Divergent Careers|Allied Health|Medical Admin|Scientist', case=False, na=False))

# Get value counts for 'Team Allocated' for rows that satisfy the condition
team_allocated_counts = df.loc[condition, 'Team Allocated'].value_counts()

# Print the value counts
print(team_allocated_counts)

Team Allocated
HS Perm    50
GP Perm    15
Name: count, dtype: int64


In [72]:
seniority_condition = df['Seniority'].str.contains('Director|DMS|Health Executive|Divergent Careers|Allied Health|Medical Admin|Scientist', case=False, na=False)
gp_perm_condition = df['Team Allocated'] == 'GP Perm'

filtered_df = df[seniority_condition & gp_perm_condition]

# Display relevant columns
filtered_df.head()

,Filed As,Status,CreatedOn,Source,Feedback Score,Country,Registration #,Registration Type,Employment Preference Attributes,Specialty,Seniority,Primary Consultant,FY24 Week,Team Allocated
10,Abdulrasheed Oluwatoyin,New Lead,2023-11-12 03:40:00,Indeed,NaN,Nigeria,NaN,NaN,Permanent,"Intensive Care (ICU), GP Emergency, Emergency ...","Divergent Careers, General Practitioner",NaN,19,GP Perm
91,BARKAT NOONARI,Ineligible,2023-07-09 00:47:00,Word of Mouth,NaN,Saudi Arabia,NaN,NaN,Permanent,"Public Health, GP Emergency, GP AMS, Emergency...","Resident (RMO), Deputy Director of Medical Ser...",NaN,1,GP Perm
259,DWARIKA PRASAD,Ineligible,2023-07-24 14:10:00,Indeed,NaN,India,NaN,NaN,Permanent,"GP Emergency, GP Sexual Health, Emergency Medi...","Health Executive, Medical Admin Registrar, Gen...",NaN,4,GP Perm
497,Lawal Tawakalitu,Ineligible,2023-08-20 23:25:00,Indeed,NaN,Nigeria,NaN,NaN,"Permanent, Telehealth","Public Health, GP Skin, General Practice","Health Executive, General Practitioner",NaN,7,GP Perm
671,Medical doctor,Ineligible,2023-09-07 18:43:00,ACEM Website,NaN,Iraq,NaN,NaN,"Permanent, Telehealth",General Practice,"Divergent Careers, General Practitioner, Consu...",NaN,10,GP Perm


## Save File

In [73]:
# Get the current date
current_date = datetime.datetime.now()

# Calculate the date of the previous Sunday
previous_sunday = current_date - timedelta(days=current_date.weekday() + 1)

# Format the date in the desired format (YYYYMMDD)
previous_sunday_str = previous_sunday.strftime("%Y%m%d")

# Create a folder if it doesn't exist
folder_name = 'cleaned_new_leads'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Construct the file path with the folder and the previous Sunday's date
file_path = os.path.join(folder_name, f'newleads_cleaned_{previous_sunday_str}.csv')

# Export DataFrame to a CSV file
df.to_csv(file_path, index=False, encoding=result['encoding'])